###Setup
Primeiro, vamos garantir que estamos usando o catálogo correto e que o schema 00_landing existe.

In [0]:
%sql
SELECT current_catalog(), current_schema()

In [0]:
%sql
USE CATALOG smart_claims_dev

In [0]:
%sql
USE SCHEMA `00_landing`

In [0]:
%sql
SELECT current_catalog(), current_schema()

###EXPLORE THE DATA SOURCE FILES

In [0]:
%sql
LIST '/Volumes/smart_claims_dev/00_landing/sql_server'

In [0]:
%python
files= dbutils.fs.ls(
'/Volumes/smart_claims_dev/00_landing/sql_server' 
)
display(files)

In [0]:
%sql
SELECT *
FROM csv.`/Volumes/smart_claims_dev/00_landing/sql_server/claims.csv`
--drop the table if it exists for demonstration

###Batch Data Ingestion with CTAs


In [0]:
%sql

SELECT *
FROM read_files(
  '/Volumes/smart_claims_dev/00_landing/sql_server/claims.csv',
  format => 'csv'
)
LIMIT 10;


In [0]:
%sql
--drop the table if it exists for demonstration purposes
DROP TABLE IF EXISTS smart_claims_dev.01_bronze.claims;
CREATE TABLE IF NOT EXISTS smart_claims_dev.`01_bronze`.claims
AS SELECT *
FROM read_files('/Volumes/smart_claims_dev/00_landing/sql_server/claims.csv',
format => 'csv'
)

In [0]:
%sql
SELECT * FROM smart_claims_dev.01_bronze.claims

In [0]:
%sql
DESCRIBE TABLE EXTENDED smart_claims_dev.01_bronze.claims;

###Python Ingestion

In [0]:
%sql
DROP TABLE IF EXISTS smart_claims_dev.01_bronze.claims;

In [0]:
%python
df = (
    spark.read
    .format("csv")
    .option("header", True)
    .load("/Volumes/smart_claims_dev/00_landing/sql_server/claims.csv")
)

df.write.mode("overwrite").saveAsTable("smart_claims_dev.01_bronze.claims")

claims_table = spark.table("smart_claims_dev.01_bronze.claims")

display(claims_table)

In [0]:
%sql
SELECT * FROM smart_claims_dev.01_bronze.claims

###COPY INTO

In [0]:
%sql
DROP TABLE IF EXISTS smart_claims_dev.01_bronze.claims;

CREATE OR REPLACE TABLE smart_claims_dev.01_bronze.claims (
    claim_no STRING,
    policy_no STRING,
    claim_date STRING,
    months_as_customer STRING,
    injury STRING,
    property STRING,
    vehicle STRING,
    total STRING,
    collision_type STRING,
    number_of_vehicles_involved STRING,
    age STRING,
    insured_relationship STRING,
    license_issue_date STRING,
    date STRING,        -- renomeei `date` para evitar conflito
    hour STRING,
    type STRING,
    severity STRING,
    number_of_witnesses STRING,
    suspicious_activity STRING
)
COMMENT 'Landing raw claims data (CSV → Bronze, raw strings)';

In [0]:
%sql
COPY INTO smart_claims_dev.01_bronze.claims
FROM '/Volumes/smart_claims_dev/00_landing/sql_server/claims.csv'
FILEFORMAT = CSV
FORMAT_OPTIONS ('header'='true')
COPY_OPTIONS ('mergeSchema'='false');


In [0]:

%python
df = spark.read.csv("/Volumes/smart_claims_dev/00_landing/sql_server/claims.csv", header=True)
print(df.columns)b

In [0]:
%sql
SELECT * FROM smart_claims_dev.01_bronze.claims